## Introduction

This notebook demonstrates how to visualize and analyze JWST MOS spectra, including how to:

* Launch [Mosviz](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html)
* [Load MOS data](https://jdaviz.readthedocs.io/en/latest/mosviz/import_data.html) from a notebook cell
* Adjust [display parameters](https://jdaviz.readthedocs.io/en/latest/cubeviz/displaycubes.html#display-settings)
* Select [spectral regions](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#spectral-regions) for further analysis 
* Identify spectral features
* Measure emission line redshifts
* [Fit spectral models](https://jdaviz.readthedocs.io/en/latest/cubeviz/plugins.html#model-fitting) to 1D spectra.




## Data

We use simulated NIRSpec MOS PRISM data created with the NIRSpec Instrument Performance Simulator (IPS).  This consists of 33 sources (primarily galaxies) plus background 
from 33 open slitlets. The dataset is not publicly released and will be replaced with another, better dataset when it becomes available.  Please do not distribute this version.

The simulated Level2a data were run through the JWST Spec2 and Spec3 pipelines to create Level 2b and Level 3 (s2d and x1d) data products. The flux calibration for this dataset is inaccurate, pending on-orbit calibration.

## Imports
* [_jdaviz_](https://jdaviz.readthedocs.io/en/latest/) : Mosviz data visualization tool
* [os]() and [glob]() for file management 
* [_astropy.utils.data_](https://docs.astropy.org/en/stable/utils/index.html) for downloading files from URLs


In [ ]:
from jdaviz import MosViz
import numpy as np
import os
import glob
from astropy.utils.data import download_file
import zipfile

## Visualize simulated NIRSpec MOS data

Execute the next cell to launch Mosviz, then follow the instructions for each task enumerated in the cell directly below the Cubeviz app.

In [ ]:
mosviz = MosViz()
mosviz.app

### UI Instructions:
#### Task 1:  Load the MOS data and view it
* Load the NIRSpec MOS spectra and image cutouts into Mosviz using the next two code cells below
* Click on the third row (not the checkbox) in the Mosviz table viewer to display data for one of the sources in the MOS dataset.
* Open the Display menu of the Image viewer. In the Layer tab, change the stretch to Logarithmic, 99 percentile.
* Try adjusting the stretch and colormap in the 2D spectrum viewer.

#### Task 2:  Toggle the slit viewer on and off
* Find the Slit Overlay Plugin by clicking the 'Lego' icon at upper right.
* Uncheck or check the box to turn the slit viewer off or on.

#### Task 3: Select a spectral region for further analysis
* Use the horizontal and vertical pan-zoom tools in the 1D spectrum viewer to zoom in on the spectral line at 2 microns.  Note that the 2D spectral region matches the selection in the 1D viewer.
* Use the spectral region selection tool in the 1D spectrum viewer to select the line at 2 microns (Subset 1).

#### Task 4: Measure line centroids using the Line Analysis plugin
* Find the Line Analysis Plugin by clicking the 'Lego' icon at upper right.
* Select a dataset with the Data dropdown (e.g. Subset1).
* Several line properties are automatically computed (flux, equivalent width, Gaussian sigma, Gaussian FWHM, and Centroid) using specutils functionality. Note that the Gaussian sigma, FWHM, and Centroid require continuum subtraction or normalization, and will not be correct if there is a significant continuum component. The equivalent width, on the other hand, requires the continuum to be normalized to unity to give a correct answer. 
* The Halpha emission line complex is at 1.92 microns (redshift z = 1.93) 
* Now determine the centroid wavelength of the line at ~1.5 microns.  What is the line ID?

#### Task 5: Identify spectral lines using the Line Lists plugin
* Open the Line Lists plugin to create a custom line list: OIII5007 (1.467 um), Halpha (1.923 um)
* Click on the 'Custom' tab and enter the first line name, wavelength, and wavelength unit, and click 'Add Line'
* In the same place, enter the values for the second line and 'Add Line'
* Click through the other spectra to see if those same lines appear (Hint: it is one of the first 5)

#### Task 6: Load a custom line list from the notebook
* Run the cell below to load a custom line list programmatically


In [ ]:
#Emission line cheat sheet
line = ['OII', 'Hbeta', 'OIIIa', 'OIIIb', 'Halpha', 'SII', 'Palpha']
wavel = [.3727, .4861, .4959, .5007, .6563, .6724, 1.8756]
z =1.93
wave = np.array(wavel)*(1+z)
print(line)
print(wave)

In [ ]:
# set the Box link and file name
output_dir='./'
#ziplink = 'https://stsci.box.com/shared/static/8iwpmppwdp9pyaun2axk7z0831esc3ox.zip'
ziplink = 'https://stsci.box.com/shared/static/xddvu98qa5k2ggjz3ru4az37t63anr1g.zip'
zipfilename = 'nirspec_mosdata_prism.zip'
if not os.path.isfile(os.path.join(output_dir, zipfilename)):
    print('Downloading {}...'.format(zipfilename))
    demo_file = download_file(ziplink, cache=True)
    # Make a symbolic link using a local name for convenience
    os.symlink(demo_file, os.path.join(output_dir, zipfilename))
else:
    print('{} already exists, skipping download...'.format(zipfilename))
    
# unzip
output_dir='./'
zipfilename='nirspec_mosdata_prism.zip'
print(output_dir+zipfilename)
zf = zipfile.ZipFile(output_dir+zipfilename, 'r')
data_dir ='mosdata'
zf.extractall(data_dir)

data_dir='mosdata'
#data_dir='mosdata_james'
# Lists of data products
slit_id = []
spectra_1d = []
spectra_2d = []
cutouts = []
for file_path in glob.glob(str(data_dir+'/*')):
    if ('x1d' in file_path):
        spectra_1d.append(file_path)
        slitid = file_path[15:20]
        #slitid = file_path[36:41]  #mosdata_james
        slit_id.append(slitid)
        #print(slitid)
    elif ('s2d' in file_path):
        spectra_2d.append(file_path)
    elif ('cutout' in file_path):
        cutouts.append(file_path)     

#Sort the slit ids and filenames
slit_id_sorted = sorted (slit_id)    
spectra_1d_sorted = sorted(spectra_1d)
spectra_2d_sorted = sorted(spectra_2d)
cutouts_sorted = sorted(cutouts)
  
#Print out the matched filenames for each slit
for s_id, f_1d, f_2d, f_cut in zip(slit_id_sorted, spectra_1d_sorted, spectra_2d_sorted, cutouts_sorted): 
    print(s_id, f_1d[8:], f_2d[14:], f_cut[8:])
    #print(s_id, f_1d[14:], f_2d[20:], f_cut[14:]) #mosdata_james
     


In [ ]:
#Load the first 10 
mosviz.load_data(spectra_1d_sorted[0:10], spectra_2d_sorted[0:10], images=cutouts_sorted[0:10])

# Or alternatively: 
#mosviz.load_metadata(cutouts_sorted[0:10])            
#mosviz.load_1d_spectra(spectra_1d_sorted[0:10])  
#mosviz.load_2d_spectra(spectra_2d_sorted[0:10]) 
#mosviz.load_images(cutouts_sorted[0:10])

If no images are provided, MOSViz can still display the spectra.

In [ ]:
mosviz_no_images = MosViz()
mosviz_no_images.app

In [ ]:
mosviz_no_images.load_data(spectra_1d_sorted[0:10], spectra_2d_sorted[0:10])

# Or alternatively: 
# mosviz_no_images.load_1d_spectra(spectra_1d)
# mosviz_no_images.load_2d_spectra(spectra_2d)

Images can be added later.

In [ ]:
mosviz_no_images.load_images(cutouts_sorted[0:10])
mosviz_no_images.load_metadata(cutouts_sorted[0:10])